# Unsupervised VAE analysis & further retraining the last layer

In [1]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
import importlib
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import optuna
import imaplib

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import umap
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score


from pretrain_neptune.get_pretrain_encoder import get_pretrain_encoder_from_modelID, get_pretrain_input_data
from pretrain.latent_task_predict import log_reg_multi_class, ridge_regression_predict


%who



2024-10-04 22:46:34.563618: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 22:46:34.580934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 22:46:34.601138: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 22:46:34.607126: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 22:46:34.622143: I tensorflow/core/platform/cpu_feature_guar

DataLoader	 LabelEncoder	 PCA	 TensorDataset	 accuracy_score	 f1_score	 get_pretrain_encoder_from_modelID	 get_pretrain_input_data	 imaplib	 
importlib	 label_encoder	 log_reg_multi_class	 ml_code_path	 nn	 np	 optim	 optuna	 os	 
pd	 plt	 precision_score	 random	 recall_score	 ridge_regression_predict	 roc_auc_score	 sns	 torch	 
umap	 


## Evaluating unsupervised models

#### getting the input data

In [2]:
# getting the input data
input_data_location='/home/leilapirhaji/PROCESSED_DATA_S_8.1.1'
(X_data_all, y_data_all, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=  get_pretrain_input_data(input_data_location)

X_data_all.shape, y_data_all.shape, X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

/home/leilapirhaji/mz_embed_engine/ml/pretrain_neptune/get_pretrain_encoder.py:72: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)
/home/leilapirhaji/mz_embed_engine/ml/pretrain_neptune/get_pretrain_encoder.py:75: DtypeWarning: Columns (1,8,14,15,24,25,26,28,29,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_train = pd.read_csv(pretrain_y_train, index_col=0)


((20548, 2736),
 (20548, 58),
 (17465, 2736),
 (17465, 58),
 (2055, 2736),
 (2055, 58),
 (1028, 2736),
 (1028, 58))

### loading the pretrain VAE models


In [3]:
import importlib
import pretrain.get_pretrain_encoder
importlib.reload(pretrain.get_pretrain_encoder)
from pretrain.get_pretrain_encoder import get_pretrain_encoder_from_local

pretrain_save_dir='/home/leilapirhaji/pretrained_models'
pretrain_name='pretrain_VAE_L_400_425_e_400_p_25_s_8.1.1'
pretrain_trial_list=[143]
pretrain_id=pretrain_trial_list[0]

(pretrain_vae, Z_train, Z_val, Z_test)=get_pretrain_encoder_from_local(pretrain_name, pretrain_id, pretrain_save_dir)
pretrain_vae

VAE(
  (encoder): Dense_Layers(
    (network): Sequential(
      (0): Linear(in_features=2736, out_features=1524, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=1524, out_features=850, bias=True)
      (4): LeakyReLU(negative_slope=0.01)
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=850, out_features=850, bias=True)
    )
  )
  (decoder): Dense_Layers(
    (network): Sequential(
      (0): Linear(in_features=425, out_features=850, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=850, out_features=1524, bias=True)
      (4): LeakyReLU(negative_slope=0.01)
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=1524, out_features=2736, bias=True)
    )
  )
)

# Re-training the last layer of unsupervised VAE models

## grid search par opt

In [ ]:
#!/bin/bash
!cd /home/leilapirhaji/mz_embed_engine/ml

# Define common variables
INPUT_DATA_LOCATION="/home/leilapirhaji/PROCESSED_DATA_S_8.1.1"
PRETRAIN_SAVE_DIR="/home/leilapirhaji/pretrained_models"
pretrain_model_name="pretrain_VAE_L_400_425_e_400_p_25_s_8.1.1"
pretrain_trial_ID="143"

TASK='BMI'


# !python ./pretrain/retrain_last_layer_pretrain_VAE_main.py \
#     --input_data_location "$INPUT_DATA_LOCATION" \
#     --pretrain_save_dir "$PRETRAIN_SAVE_DIR" \
#     --pretrain_model_name "$pretrain_model_name" \
#     --pretrain_trial_ID "$pretrain_trial_ID" \
#     --task "$TASK" \
#     --add_post_latent_layers 'False' \
#     --post_latent_layer_size "64" \
#     --num_layers_to_retrain "1" \
#     --dropout_rate 0.4 \
#     --learning_rate 1e-5 1e-4 \
#     --l1_reg 1e-6 \
#     --weight_decay 1e-3 \
#     --batch_size 32 \
#     --patience 0 \
#     --num_epochs 20

In [19]:
#open the results of grid search
#reading the output
task=TASK.replace(' ','_')

result_path=f'{PRETRAIN_SAVE_DIR}/{pretrain_model_name}/trial_{pretrain_trial_ID}/{task}'

#best paramteres
with open(f'{result_path}/{task}_best_params_grid_search.txt', 'r') as f:
    best_params = f.read()        

#best model results
best_model_results=pd.read_csv(f'{result_path}/{task}_best_model_results_grid_search.csv')
    
#All grid search results
all_grid_search_results= pd.read_csv(f'{result_path}/{task}_all_grid_search_results.csv')
#all_grid_search_results.sort_values('val_metric_TL', ascending=False, inplace=True)
# Convert a column (e.g., column 'B') to scientific notation
all_grid_search_results['learning_rate'] = all_grid_search_results['learning_rate'].apply(lambda x: '{:.2e}'.format(x))

all_grid_search_results

,add_post_latent_layers,post_latent_layer_size,num_layers_to_retrain,dropout,learning_rate,l1_reg_weight,l2_reg_weight,batch_size,patience,num_epochs,val_c_index_TL,val_c_index_noTL
0,False,64,1,0.4,1.00e-06,0.000001,0.001,32,0,20,0.724059,0.674472
1,False,64,1,0.4,3.00e-06,0.000001,0.001,32,0,20,0.753444,0.724059
2,False,64,1,0.4,5.00e-06,0.000001,0.001,32,0,20,0.760790,0.734160
3,False,64,1,0.4,7.00e-06,0.000001,0.001,32,0,20,0.765381,0.742883
4,False,64,1,0.4,9.00e-06,0.000001,0.001,32,0,20,0.768136,0.742424
5,False,64,1,0.4,1.00e-05,0.000001,0.001,32,0,20,0.768595,0.742883
6,False,64,1,0.4,3.00e-05,0.000001,0.001,32,0,20,0.773186,0.749311
7,False,64,1,0.4,5.00e-05,0.000001,0.001,32,0,20,0.770432,0.752066
8,False,64,1,0.4,7.00e-05,0.000001,0.001,32,0,20,0.769513,0.752525
9,False,64,1,0.4,9.00e-05,0.000001,0.001,32,0,20,0.767218,0.752525


In [20]:
best_model_results

,Unnamed: 0,Model,Dataset,C-index
0,0,Transfer Learning,Train,0.749240
1,1,Transfer Learning,Validation,0.773186
2,2,Transfer Learning,Test,0.658278
3,3,No Transfer Learning,Train,0.706099
4,4,No Transfer Learning,Validation,0.749311
5,5,No Transfer Learning,Test,0.657837


In [114]:
best_params

"{'add_post_latent_layers': False, 'post_latent_layer_size': 64, 'num_layers_to_retrain': 2, 'dropout': 0.4, 'learning_rate': 0.0001, 'l1_reg_weight': 1e-06, 'l2_reg_weight': 0.001, 'batch_size': 32, 'patience': 0, 'num_epochs': 20}"

### Plotting the latnet space

In [ ]:
import pretrain.get_pretrain_encoder
importlib.reload(pretrain.get_pretrain_encoder)
from pretrain.get_pretrain_encoder import visualize_latent_space_multiple_tasks

model_id=f'trial_{pretrain_id}'

visualize_latent_space_multiple_tasks( model_id, model_folder, Z_all, Z_train, Z_val, Z_test, y_data_all, y_data_train, y_data_val, y_data_test, method='UMAP', n_components=2, dpi=300, palette='tab10')